In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pickle
import time
import re


In [2]:

# Setup Chrome driver

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)
# # Path to your ChromeDriver (sesuaikan dengan lokasi driver di komputermu)
# chrome_driver_path = "/path/to/chromedriver"

# # Buat instance dari driver Chrome
# driver = webdriver.Chrome(executable_path=chrome_driver_path)

In [3]:

# Buka halaman login Sinta Kemdikbud
driver.get("https://sinta.kemdikbud.go.id/logins")


In [4]:

# Tunggu beberapa detik agar halaman login terbuka
time.sleep(3)
username=""
password=""


In [5]:
# Isi form login (sesuaikan dengan elemen yang ada pada HTML)
username_field = driver.find_element(By.NAME, "username") if len(driver.find_elements(By.NAME, "username")) > 0 else driver.find_element(By.XPATH, '//input[@name="username"]')
password_field = driver.find_element(By.NAME, "password") if len(driver.find_elements(By.NAME, "password")) > 0 else driver.find_element(By.XPATH, '//input[@name="password"]')

# Masukkan kredensial login (ganti dengan username dan password yang benar)
username_field.send_keys(username)
print(f"Input username: {username}")
password_field.send_keys(password)
print(f"Input password: {password}")

Input username: syahroni@widyagama.ac.id
Input password: @N3002honh


In [6]:
# Tekan tombol login
login_button = driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-block.btn-info')
login_button.click()

In [7]:
# Tunggu beberapa detik agar login diproses
time.sleep(5)

# Simpan sesi login menggunakan pickle
with open("sinta_session.pkl", "wb") as session_file:
    pickle.dump(driver.get_cookies(), session_file)

print("Session saved successfully!")


Session saved successfully!


In [9]:

# Tutup browser
# driver.quit()

# Untuk membuka sesi yang tersimpan dan melanjutkan sesi sebelumnya
# Load sesi

# driver = webdriver.Chrome(executable_path=chrome_driver_path)
# driver.get("https://sinta.kemdikbud.go.id/")

# Load cookies yang sudah disimpan
with open("sinta_session.pkl", "rb") as session_file:
    cookies = pickle.load(session_file)
    for cookie in cookies:
        driver.add_cookie(cookie)

# Refresh halaman untuk memastikan sesi aktif
driver.refresh()

print("Session loaded successfully!")


Session loaded successfully!


In [10]:

# Navigasi ke halaman profil peneliti
researcher_id = "5986966"  # Ganti dengan ID peneliti yang sesuai
driver.get(f"https://sinta.kemdikbud.go.id/authors/profile/{researcher_id}/")

print(f"Navigated to researcher profile with ID: {researcher_id}")

Navigated to researcher profile with ID: 5986966


In [14]:
# Navigasi ke halaman profil peneliti pada tab Scopus
researcher_id = "5986966"  # Ganti dengan ID peneliti yang sesuai
driver.get(f"https://sinta.kemdikbud.go.id/authors/profile/{researcher_id}/?view=scopus")

print(f"Navigated to Scopus tab for researcher profile with ID: {researcher_id}")

Navigated to Scopus tab for researcher profile with ID: 5986966


In [13]:
# Tunggu beberapa detik agar halaman profil peneliti termuat sepenuhnya
time.sleep(3)

# Mendapatkan elemen pagination
pagination_items = driver.find_elements(By.CSS_SELECTOR, 'nav[aria-label="pagination-sample"] ul.pagination li.page-item')

# Menampilkan informasi pagination
for item in pagination_items:
    class_name = item.get_attribute("class")
    link = item.find_element(By.TAG_NAME, "a").get_attribute("href") if "disabled" not in class_name and "active" not in class_name else None
    text = item.text
    print(f"Page Item: {text}, Link: {link}")

# Mendapatkan informasi jumlah halaman dari teks pagination
pagination_text_element = driver.find_element(By.CSS_SELECTOR, 'div.pagination-text')
pagination_text = pagination_text_element.text
match = re.search(r'Page \d+ of (\d+)', pagination_text)
if match:
    total_pages = int(match.group(1))
    print(f"Total number of pages: {total_pages}")
else:
    print("Unable to determine total number of pages.")

Page Item: Previous, Link: None
Page Item: 1, Link: None
Page Item: 2, Link: https://sinta.kemdikbud.go.id/authors/profile/5986966/?page=2&view=scopus
Page Item: Next, Link: https://sinta.kemdikbud.go.id/authors/profile/5986966/?page=2&view=scopus
Total number of pages: 2


In [20]:
# Mendapatkan jumlah halaman dari teks pagination
pagination_text_element = driver.find_element(By.CSS_SELECTOR, 'div.pagination-text')
pagination_text = pagination_text_element.text
match = re.search(r'Page \d+ of (\d+)', pagination_text)
if match:
    total_pages = int(match.group(1))
    print(f"Total number of pages: {total_pages}")
else:
    print("Unable to determine total number of pages.")
    total_pages = 1


Total number of pages: 2


In [21]:
# Loop untuk mengambil data di semua halaman
for page in range(1, total_pages + 1):
    print(f"Scraping page {page} of {total_pages}")
    
    # Tunggu agar halaman termuat
    time.sleep(3)
    
    # Mendapatkan informasi artikel Scopus
    div_elements = driver.find_elements(By.CSS_SELECTOR, 'div.ar-list-item.mb-5')
    for div in div_elements:
        try:
            title = div.find_element(By.CSS_SELECTOR, 'div.ar-title > a').text
        except:
            title = "N/A"
        try:
            journal = div.find_element(By.CSS_SELECTOR, 'a.ar-pub').text
        except:
            journal = "N/A"
        try:
            year = div.find_element(By.CSS_SELECTOR, 'a.ar-year').text
        except:
            year = "N/A"
        try:
            link = div.find_element(By.CSS_SELECTOR, 'div.ar-title > a').get_attribute("href")
        except:
            link = "N/A"
        
        articles.append({
            "Title": title,
            "Journal": journal,
            "Year": year,
            "Link": link
        })
    
    # Jika masih ada halaman berikutnya, klik tombol "Next"
    if page < total_pages:
        try:
            next_button = driver.find_element(By.LINK_TEXT, "Next")
            next_button.click()
        except:
            print("No next button found, stopping navigation.")
            break


Scraping page 1 of 2
Scraping page 2 of 2
                                                Title  \
0   Effects of overloading commercial vehicles on ...   
1   Identification of Road Surface Defects Using M...   
2   Moving Car Observation (MCO) for Road Surface ...   
3   Smart Route Choice Based on Google Maps Applic...   
4   Correlation of Expansive Soil and Road Pavemen...   
5   Application of the Synthetic Over-Sampling Met...   
6   Empirical Study of a Spatial Analysis for Pron...   
7   ACCIDENT RISK ANALYSIS OF ROAD GEOMETRIC COMPO...   
8   ANALYSIS OF INTERCITY BUS PUBLIC TRANSPORT SAF...   
9   Analysis of bus performance on the risk of tra...   
10  Effects of overloading commercial vehicles on ...   
11  Identification of Road Surface Defects Using M...   
12  Moving Car Observation (MCO) for Road Surface ...   
13  Smart Route Choice Based on Google Maps Applic...   
14  Correlation of Expansive Soil and Road Pavemen...   
15  Application of the Synthetic Over-Sampling

In [22]:
# Konversi artikel menjadi DataFrame
articles_df = pd.DataFrame(articles)

# Menyimpan DataFrame ke dalam CSV dan Excel
articles_df.to_csv("scopus_articles.csv", index=False)
articles_df.to_excel("scopus_articles.xlsx", index=False)

In [23]:
# Menampilkan DataFrame
print(articles_df)

                                                Title  \
0   Effects of overloading commercial vehicles on ...   
1   Identification of Road Surface Defects Using M...   
2   Moving Car Observation (MCO) for Road Surface ...   
3   Smart Route Choice Based on Google Maps Applic...   
4   Correlation of Expansive Soil and Road Pavemen...   
5   Application of the Synthetic Over-Sampling Met...   
6   Empirical Study of a Spatial Analysis for Pron...   
7   ACCIDENT RISK ANALYSIS OF ROAD GEOMETRIC COMPO...   
8   ANALYSIS OF INTERCITY BUS PUBLIC TRANSPORT SAF...   
9   Analysis of bus performance on the risk of tra...   
10  Effects of overloading commercial vehicles on ...   
11  Identification of Road Surface Defects Using M...   
12  Moving Car Observation (MCO) for Road Surface ...   
13  Smart Route Choice Based on Google Maps Applic...   
14  Correlation of Expansive Soil and Road Pavemen...   
15  Application of the Synthetic Over-Sampling Met...   
16  Empirical Study of a Spatia

In [19]:
import pandas as pd
# Mendapatkan informasi artikel Scopus
div_elements = driver.find_elements(By.CSS_SELECTOR, 'div.ar-list-item.mb-5')
articles = []

for div in div_elements:
    try:
        title = div.find_element(By.CSS_SELECTOR, 'div.ar-title > a').text
    except:
        title = "N/A"
    try:
        journal = div.find_element(By.CSS_SELECTOR, 'a.ar-pub').text
    except:
        journal = "N/A"
    try:
        year = div.find_element(By.CSS_SELECTOR, 'a.ar-year').text
    except:
        year = "N/A"
    try:
        link = div.find_element(By.CSS_SELECTOR, 'div.ar-title > a').get_attribute("href")
    except:
        link = "N/A"
    
    articles.append({
        "Title": title,
        "Journal": journal,
        "Year": year,
        "Link": link
    })
    
# Konversi artikel menjadi DataFrame
articles_df = pd.DataFrame(articles)

# Menampilkan DataFrame
print(articles_df)

                                               Title  \
0  Effects of overloading commercial vehicles on ...   
1  Identification of Road Surface Defects Using M...   
2  Moving Car Observation (MCO) for Road Surface ...   
3  Smart Route Choice Based on Google Maps Applic...   
4  Correlation of Expansive Soil and Road Pavemen...   
5  Application of the Synthetic Over-Sampling Met...   
6  Empirical Study of a Spatial Analysis for Pron...   
7  ACCIDENT RISK ANALYSIS OF ROAD GEOMETRIC COMPO...   
8  ANALYSIS OF INTERCITY BUS PUBLIC TRANSPORT SAF...   
9  Analysis of bus performance on the risk of tra...   

                                             Journal  Year  \
0                         AIP Conference Proceedings  2024   
1  2023 4th International Conference on Artificia...  2023   
2  ICCoSITE 2023 - International Conference on Co...  2023   
3  Proceedings - 2022 1st International Conferenc...  2022   
4  Proceedings - 2022 IEEE International Conferen...  2022   
5  Internat

In [24]:
def navigate_to_tab(driver, researcher_id, view):
    driver.get(f"https://sinta.kemdikbud.go.id/authors/profile/{researcher_id}/?view={view}")
    print(f"Navigated to {view} tab for researcher profile with ID: {researcher_id}")
    time.sleep(3)

In [30]:
def scrape_articles(driver):
    articles = []
    pagination_text_element = driver.find_element(By.CSS_SELECTOR, 'div.pagination-text')
    pagination_text = pagination_text_element.text
    match = re.search(r'Page \d+ of (\d+)', pagination_text)
    if match:
        total_pages = int(match.group(1))
        print(f"Total number of pages: {total_pages}")
    else:
        print("Unable to determine total number of pages.")
        total_pages = 1

    for page in range(1, total_pages + 1):
        print(f"Scraping page {page} of {total_pages}")
        time.sleep(3)
        div_elements = driver.find_elements(By.CSS_SELECTOR, 'div.ar-list-item.mb-5')
        for div in div_elements:
            try:
                title = div.find_element(By.CSS_SELECTOR, 'div.ar-title > a').text
            except:
                title = "N/A"
            try:
                link = div.find_element(By.CSS_SELECTOR, 'div.ar-title > a').get_attribute("href")
            except:
                link = "N/A"
            try:
                quartile = div.find_element(By.CSS_SELECTOR, 'a.ar-quartile').text
            except:
                quartile = "N/A"
            try:
                journal = div.find_elements(By.CSS_SELECTOR, 'a.ar-pub')[-1].text if div.find_elements(By.CSS_SELECTOR, 'a.ar-pub') else "N/A"
            except:
                journal = "N/A"
            try:
                year = div.find_element(By.CSS_SELECTOR, 'a.ar-year').text
            except:
                year = "N/A"
            try:
                cited_count = div.find_element(By.CSS_SELECTOR, 'a.ar-cited').text
            except:
                cited_count = "N/A"
            try:
                author_order = div.find_element(By.XPATH, '//a[contains(text(), "Author Order")]').text
            except:
                author_order = "N/A"
            try:
                authors = div.find_element(By.XPATH, '//a[contains(text(), "Authors")]').text
            except:
                authors = "N/A"
            try:
                scopus_indexed = div.find_element(By.CSS_SELECTOR, 'span.num-stat.scopus-indexed').text
            except:
                scopus_indexed = "N/A"
            try:
                doi = div.find_element(By.CSS_SELECTOR, 'a.ar-sinta').text
            except:
                doi = "N/A"
            
            articles.append({
                "Title": title,
                "Link": link,
                "Quartile": quartile,
                "Journal": journal,
                "Year": year,
                "Cited Count": cited_count,
                "Author Order": author_order,
                "Authors": authors,
                "Scopus Indexed": scopus_indexed,
                "DOI": doi
            })
        
        if page < total_pages:
            try:
                next_button = driver.find_element(By.LINK_TEXT, "Next")
                next_button.click()
            except:
                print("No next button found, stopping navigation.")
                break
    
    return articles

In [31]:

def save_articles_to_files(articles, filename_prefix):
    articles_df = pd.DataFrame(articles)
    articles_df.to_csv(f"{filename_prefix}_articles.csv", index=False)
    articles_df.to_excel(f"{filename_prefix}_articles.xlsx", index=False)
    print(articles_df)

In [32]:
  # Scrape Scopus Articles
navigate_to_tab(driver, researcher_id, "scopus")
scopus_articles = scrape_articles(driver)
save_articles_to_files(scopus_articles, "scopus")

# Scrape WOS Articles
navigate_to_tab(driver, researcher_id, "wos")
wos_articles = scrape_articles(driver)
save_articles_to_files(wos_articles, "wos")

Navigated to scopus tab for researcher profile with ID: 5986966
Total number of pages: 2
Scraping page 1 of 2
Scraping page 2 of 2
                                                Title  \
0   Effects of overloading commercial vehicles on ...   
1   Identification of Road Surface Defects Using M...   
2   Moving Car Observation (MCO) for Road Surface ...   
3   Smart Route Choice Based on Google Maps Applic...   
4   Correlation of Expansive Soil and Road Pavemen...   
5   Application of the Synthetic Over-Sampling Met...   
6   Empirical Study of a Spatial Analysis for Pron...   
7   ACCIDENT RISK ANALYSIS OF ROAD GEOMETRIC COMPO...   
8   ANALYSIS OF SOIL CHARACTERISTICS ON EXPANSIVE ...   
9   ANALYSIS OF INTERCITY BUS PUBLIC TRANSPORT SAF...   
10  Analysis of bus performance on the risk of tra...   
11  Driver performance problems of intercity bus p...   

                                                 Link  \
0   https://www.scopus.com/record/display.uri?eid=...   
1   https://w